In [3]:
# import the required libraries
import itertools
import cv2
import numpy as np
import random
import os
from PIL import Image
import time

# load the yoloV3 weights
model = cv2.dnn.readNet("D://PROJECTS/social distance/yolov3.weights", "D://PROJECTS/social distance/yolov3.cfg")

# give the threshold distance
distance_thres = 100

#give the input video
cap = cv2.VideoCapture('D://PROJECTS/social distance/pedestrians.mp4')

#apply the formula to calculate the distance between people
def dist(x,y):
    try:
        return ((x[0]-y[0])**2 + (x[1]-y[1])**2)**0.5
    except:
        return

layer_names = model.getLayerNames()
output_layers = [layer_names[i - 1] for i in model.getUnconnectedOutLayers()]
print('OUTPUT_LAYERS',output_layers)

_,frame = cap.read()
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*"MJPG")
writer = cv2.VideoWriter('D://PROJECTS/social distance/social_distance_unoptimized_version.avi', fourcc, 30,(width,height), True)

# used to record the time when we processed last frame
prev_frame_time = 0
  
# used to record the time at which we processed current frame
new_frame_time = 0

# go through the frame by frame in a video
ret = True
while ret:

    ret, img = cap.read()           # here img is one  frame 
    if ret:
        height, width = img.shape[:2]

        blob = cv2.dnn.blobFromImage(img, 0.00392, (416,416), (0, 0, 0), True, crop=False)

        model.setInput(blob)
        outs = model.forward(output_layers)

        confidences = []
        boxes = []
        
        for out in outs:
            for detection in out:
                scores = detection[5:]
                class_id = np.argmax(scores)
                if class_id!=0:
                    continue
                confidence = scores[class_id]
                if confidence > 0.2:
                    center_x = int(detection[0] * width)
                    center_y = int(detection[1] * height)

                    w = int(detection[2] * width)
                    h = int(detection[3] * height)
                    x = int(center_x - w / 2)
                    y = int(center_y - h / 2)

                    boxes.append([x, y, w, h])
                    confidences.append(float(confidence))
       
        
        indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.4, 0.3)
        new_frame_time = time.time()
        fps = 1/(new_frame_time-prev_frame_time)
        prev_frame_time = new_frame_time
        fps = int(fps)
        fps = str(fps)

        persons = []
        person_centres = []
        violate = set()

        for i in range(len(boxes)):
            if i in indexes:
                x,y,w,h = boxes[i]
                persons.append(boxes[i])
                person_centres.append([x+w//2,y+h//2])

        for i in range(len(persons)):
            for j in range(i+1,len(persons)):
                if dist(person_centres[i],person_centres[j]) <= distance_thres:  # checking the violation condition
                    violate.add(tuple(persons[i]))
                    violate.add(tuple(persons[j]))

        v = 0
        for (x,y,w,h) in persons:
            if (x,y,w,h) in violate:
                color = (0,0,225)
                v+=1
            else:
                
                color = (0,255,0)

            cv2.rectangle(img,(x,y),(x+w,y+h),color,2)
            cv2.circle(img,(x+w//2,y+h//2),2,(0,0,255),2)
         
        
        
        cv2.putText(img,'No of Violations : '+str(v),(80,frame.shape[0]-10),cv2.FONT_HERSHEY_SIMPLEX,4,(0,0,255),4)
    
        cv2.putText(img,"FPS :"+ fps, (7,90), cv2.FONT_HERSHEY_SIMPLEX, 4, (255,0,0), 4, cv2.LINE_AA)
    
        writer.write(img)
        cv2.imshow("SOCIAL DISTANCE", img)
        
    if cv2.waitKey(1) == 27:
        break

cap.release()
cv2.destroyAllWindows()


OUTPUT_LAYERS ['yolo_82', 'yolo_94', 'yolo_106']
